In [1]:
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input, concatenate,Layer, Conv1D, Flatten
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.mixed_precision import set_global_policy

from sklearn.model_selection import train_test_split
tf.config.optimizer.set_jit(False)
tf.keras.backend.clear_session()
# 混合精度
set_global_policy('mixed_float16')
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
# 定义牌的类型和范围
type_encoding = {'?':0, 'moqie':1, 'chipenggang':2}
suits = [
    '1m','2m','3m','4m','5m','6m','7m','8m','9m',
    '1s','2s','3s','4s','5s','6s','7s','8s','9s',
    '1p','2p','3p','4p','5p','6p','7p','8p','9p',
    '1z','2z','3z','4z','5z','6z','7z'
]
# 初始化编码字典
tile_encoding = {}
index = 0
# 生成摸入-切出组合
for suit in suits:
    action = f"{suit}"  # 组合形式，例如 '1m_1m'
    tile_encoding[action] = index
    index += 1
print(tile_encoding)

def encode_to_onehot(tile_in,tile_out,max_len=18):
    tile_in = ['5' + tile[1:] if tile in ['0m', '0s', '0p'] else tile for tile in tile_in]
    tile_out = ['5' + tile[1:] if tile in ['0m', '0s', '0p'] else tile for tile in tile_out]

    types =  [type_encoding[f'{in_tile}'] for in_tile in tile_in]
    tiles =  [tile_encoding[f'{out_tile}'] for out_tile in tile_out]
    onehot1 = np.zeros((max_len, len(type_encoding)))  # 序列长度为 max_len
    for i, idx in enumerate(types):
        if i >= max_len:
            break
        onehot1[i, idx] = 1
    onehot2 = np.zeros((max_len, len(tile_encoding)))  # 序列长度为 max_len
    for i, idx in enumerate(tiles):
        if i >= max_len:
            break
        onehot2[i, idx] = 1
    return onehot1, onehot2
def preprocess_data(data):
    X_out=[]
    X_in=[]
    y=[]
    for entry in data["results"]:
        onehot1, onehot2=encode_to_onehot(entry['tile_in'], entry['tile_out'])
        X_in.append(onehot1)
        X_out.append(onehot2)
        y.append(1 if entry['tingpai'] else 0)
    X_out = np.array(X_out)
    X_in = np.array(X_in)
    y = np.array(y)
    return X_out,X_in,y

print("开始读取JSON数据")
with open('output', 'r') as file:
    data = json.load(file)
print("开始预处理数据")
X_out,X_in,y=preprocess_data(data)



2024-10-02 11:16:32.988880: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-02 11:16:33.000599: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-02 11:16:33.004127: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-02 11:16:33.013467: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-02 11:16:33.653520: W tensorflow/compiler/tf2

Num GPUs Available:  0
{'1m': 0, '2m': 1, '3m': 2, '4m': 3, '5m': 4, '6m': 5, '7m': 6, '8m': 7, '9m': 8, '1s': 9, '2s': 10, '3s': 11, '4s': 12, '5s': 13, '6s': 14, '7s': 15, '8s': 16, '9s': 17, '1p': 18, '2p': 19, '3p': 20, '4p': 21, '5p': 22, '6p': 23, '7p': 24, '8p': 25, '9p': 26, '1z': 27, '2z': 28, '3z': 29, '4z': 30, '5z': 31, '6z': 32, '7z': 33}
开始读取JSON数据
开始预处理数据


In [3]:
# 定义一个简单的注意力机制层
class Attention(Layer):
    def __init__(self):
        super(Attention, self).__init__()

    def call(self, inputs):
        # 输入: (batch_size, timesteps, units)
        score = tf.matmul(inputs, inputs, transpose_b=True)  # 计算相似性分数
        attention_weights = tf.nn.softmax(score, axis=-1)  # 计算权重
        context_vector = tf.matmul(attention_weights, inputs)  # 根据权重计算上下文向量
        return context_vector
# 构建LSTM模型
def build_model(input_shape,input_shape2):
    input1 = Input(shape=input_shape)
    input2 = Input(shape=input_shape2)
    x1 = LSTM(64, return_sequences=True)(input1)
    x1 = Dropout(0.2)(x1)
    x1 = LSTM(32)(x1)

    x2 = LSTM(64, return_sequences=True)(input2)
    x2 = Dropout(0.2)(x2)
    x2 = LSTM(32)(x2)

    merged = concatenate([x1, x2])
    output = Dense(1, activation='sigmoid')(merged)

    model = Model(inputs=[input1, input2], outputs=output)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# 划分训练集和测试集
X_train_in, X_test_in, X_train_out, X_test_out, y_train, y_test  = train_test_split(X_in,X_out, y, test_size=0.1, random_state=42)

input_shape_in = (X_in.shape[1], X_in.shape[2])
input_shape_out = (X_out.shape[1], X_out.shape[2])
model = build_model(input_shape_in, input_shape_out)

print("开始训练模型")
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6)
model.fit([X_train_in, X_train_out], y_train, epochs=30, batch_size=16, validation_data=([X_test_in, X_test_out], y_test), callbacks=[reduce_lr])
# model.fit([X_train_in, X_train_out], y_train, epochs=30, batch_size=16, validation_data=([X_test_in, X_test_out], y_test))
# early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
# model.fit(X_train, y_train, epochs=30, batch_size=16, validation_data=(X_test, y_test), callbacks=[early_stopping])


开始训练模型
Epoch 1/30
4527/4527 ━━━━━━━━━━━━━━━━━━━━ 49s 10ms/step - accuracy: 0.8707 - loss: 0.3113 - val_accuracy: 0.8820 - val_loss: 0.2811 - learning_rate: 0.0010
Epoch 2/30
4527/4527 ━━━━━━━━━━━━━━━━━━━━ 40s 9ms/step - accuracy: 0.8820 - loss: 0.2786 - val_accuracy: 0.8895 - val_loss: 0.2713 - learning_rate: 0.0010
Epoch 3/30
4527/4527 ━━━━━━━━━━━━━━━━━━━━ 40s 9ms/step - accuracy: 0.8842 - loss: 0.2755 - val_accuracy: 0.8913 - val_loss: 0.2633 - learning_rate: 0.0010
Epoch 4/30
4527/4527 ━━━━━━━━━━━━━━━━━━━━ 40s 9ms/step - accuracy: 0.8931 - loss: 0.2558 - val_accuracy: 0.9012 - val_loss: 0.2507 - learning_rate: 0.0010
Epoch 5/30
4527/4527 ━━━━━━━━━━━━━━━━━━━━ 39s 9ms/step - accuracy: 0.9023 - loss: 0.2425 - val_accuracy: 0.9082 - val_loss: 0.2366 - learning_rate: 0.0010
Epoch 6/30
4527/4527 ━━━━━━━━━━━━━━━━━━━━ 40s 9ms/step - accuracy: 0.9121 - loss: 0.2203 - val_accuracy: 0.9133 - val_loss: 0.2303 - learning_rate: 0.0010
Epoch 7/30
4527/4527 ━━━━━━━━━━━━━━━━━━━━ 42s 9ms/step - accur

In [28]:
# 正式读入测试数据
with open('pa.json', 'r') as file:
    data = json.load(file)
X_test_out,X_test_in,y_test=preprocess_data(data)
print(y)
test_loss, test_acc = model.evaluate([X_test_in, X_test_out], y_test)
y_pred = model.predict([X_test_in, X_test_out])
y_pred_classes = (y_pred > 0.5).astype(int)
print(y_pred_classes,y_test)

print(f"Test Accuracy: {test_acc}")

model.save('my_model.keras')

[0 0 0 ... 0 0 0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 832ms/step - accuracy: 0.2500 - loss: 3.4676
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
[[1]
 [0]
 [1]
 [1]] [0 0 0 0]
Test Accuracy: 0.25


In [29]:
import json
from keras.models import load_model
model = load_model('my_model.keras')

# 正式读入测试数据
with open('pa.json', 'r') as file:
    data = json.load(file)
X_test_out,X_test_in,y_test=preprocess_data(data)
y_pred = model.predict([X_test_in, X_test_out])
y_pred = model.predict([X_test_in, X_test_out])
y_pred_classes = (y_pred > 0.5).astype(int)
print("Probility:", y_pred)
print("Predicted classes:", y_pred_classes)
# print("Actual y_test:", y_test)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 360ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Probility: [[9.990e-01]
 [3.397e-05]
 [9.907e-01]
 [8.955e-01]]
Predicted classes: [[1]
 [0]
 [1]
 [1]]
